In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os


In [2]:
df_obs = pd.read_csv("/Net/Groups/BSI/work_scratch/ppapastefanou/atto_summerschool_25/data/ATTO_evaluation.csv")

In [13]:
base_path = '/Net/Groups/BSI/scratch/ppapastefanou/simulations/QPy/ATTO_example_03'
base_path_output = os.path.join(base_path, "output")
subdirs = [int(d) for d in os.listdir(base_path_output) if os.path.isdir(os.path.join(base_path_output, d))]
subdirs.sort()

if not subdirs:
    print("Empty output. Stopping processing")
    
rmses_year_day = np.zeros(len(subdirs))
rmses_day = np.zeros(len(subdirs))

for out in subdirs:
    ds_mod_ass = xr.open_dataset(os.path.join(base_path_output, str(out),"Q_ASSIMI_fluxnetdata_timestep.nc"))
    ds_mod_veg = xr.open_dataset(os.path.join(base_path_output,str(out), "VEG_fluxnetdata_timestep.nc"))
    ds_mod_sb  = xr.open_dataset(os.path.join(base_path_output,str(out), "SB_fluxnetdata_timestep.nc"))
    
    
    df_mod = ds_mod_veg['npp_avg'].to_pandas().to_frame()
    df_mod['gpp_avg'] = ds_mod_ass['gpp_avg']
    df_mod['nee_avg'] = -(ds_mod_veg['npp_avg'] -ds_mod_sb['het_respiration_avg'] )
    
    
    df_mod['Year'] = df_mod.index.year
    df_mod['day_of_year_adj'] = df_mod.index.dayofyear
    df_mod['Hour'] = df_mod.index.hour
    df_mod['Minute'] = df_mod.index.minute
    
    
    df_merged = pd.merge(
    df_mod,
    df_obs,
    on=["Year", "day_of_year_adj", "Hour", "Minute"],
    how="inner"   # or "outer", "left", "right" depending on your need
                        )

    df_merged['datetime'] = (
        pd.to_datetime(df_merged['Year'].astype(str), format='%Y')  # start of the year
        + pd.to_timedelta(df_merged['day_of_year_adj'], unit='D')  # adjust day of year
        + pd.to_timedelta(df_merged['Hour'], unit='h')
        + pd.to_timedelta(df_merged['Minute'], unit='m')
    )
    df_merged.set_index('datetime', inplace=True)
    
    
    dfm = df_merged.groupby([df_merged.index.day_of_year, df_merged.index.year]).agg(
        nee_mean_mod=("nee_avg", "mean"),
        nee_q25_mod=("nee_avg", lambda x: x.quantile(0.25)),
        nee_median_mod=("nee_avg", "median"),
        nee_q75_mod=("nee_avg", lambda x: x.quantile(0.75)),
        
        nee_mean_obs=("NEE_U50_orig", "mean"),
        nee_q25_obs=("NEE_U50_orig", lambda x: x.quantile(0.25)),
        nee_median_obs=("NEE_U50_orig", "median"),
        nee_q75_obs=("NEE_U50_orig", lambda x: x.quantile(0.75)),
    )


    vrmse_doy = rmse(dfm['nee_mean_mod'], dfm['nee_mean_obs'])
    rmses_day[out] = vrmse_doy
    
    
    dfm = df_merged.groupby([df_merged.index.day_of_year]).agg(
        nee_mean_mod=("nee_avg", "mean"),
        nee_q25_mod=("nee_avg", lambda x: x.quantile(0.25)),
        nee_median_mod=("nee_avg", "median"),
        nee_q75_mod=("nee_avg", lambda x: x.quantile(0.75)),
        
        nee_mean_obs=("NEE_U50_orig", "mean"),
        nee_q25_obs=("NEE_U50_orig", lambda x: x.quantile(0.25)),
        nee_median_obs=("NEE_U50_orig", "median"),
        nee_q75_obs=("NEE_U50_orig", lambda x: x.quantile(0.75)),
    )


    vrmse_doy = rmse(dfm['nee_mean_mod'], dfm['nee_mean_obs'])
    rmses_year_day[out] = vrmse_doy
    
    print(f"{out} {vrmse_doy}")

    ds_mod_ass.close()
    ds_mod_sb.close()
    ds_mod_veg.close()

0 1.292764449397514
1 1.4049967948759448
2 1.4172603250638833
3 1.3162961788863243
4 1.3528950125664398
5 1.36453099690492
6 1.3595010866157562
7 1.3224037511199471
8 1.3995290719576567
9 1.3776918156902096
10 1.3475283531329414
11 1.3556818978717704
12 1.3846065953151951
13 1.284170351961197
14 1.3448244266596796
15 1.3883197436310035
16 1.3630616607490396
17 1.2614928216986445
18 1.3390086112474906
19 1.3620299807017437
20 1.470339923380599
21 1.3829004155815532
22 1.5432482388135034
23 1.3946449461903465
24 1.368557744746667
25 1.372118407863609
26 1.3628197275127807
27 1.3993684910685966
28 1.307933271321836
29 1.379756279116368
30 1.429453054887459
31 1.3756380347536332
32 1.3035964283773647
33 1.3583719554078437
34 1.4086875570685242
35 1.3740864938032007
36 1.2759427518669606
37 1.4017005773810818
38 1.399434909024682
39 1.3555101124364493
40 1.365062480640742
41 1.4020012810610376
42 1.3496060387387663
43 1.359839732611224
44 1.3314120414165864
45 1.3026412751297394
46 1.403666

In [ ]:
df_param = pd.read_csv(os.path.join(base_path, "parameters.csv"))
df_param['rmse_year_doy'] = rmses_year_day
df_param['rmse_doy'] = rmses_day
os.makedirs(os.path.join(base_path, "post_process"),exist_ok=True)
df_param.to_csv(os.path.join(base_path, "post_process", "parameters_best.csv"))

In [16]:
df_param

,sand_fracs,id,fid,clay_fracs,fresp,rmse_year_doy,rmse_doy
0,0.233,0,0,0.390,0.294,1.292764,3.783984
1,0.256,1,1,0.598,0.207,1.404997,3.880258
2,0.389,2,2,0.521,0.165,1.417260,3.890917
3,0.273,3,3,0.384,0.312,1.316296,3.804736
4,0.276,4,4,0.477,0.181,1.352895,3.836237
...,...,...,...,...,...,...,...
115,0.347,115,115,0.447,0.249,1.374891,3.855223
116,0.384,116,116,0.509,0.203,1.405593,3.881212
117,0.365,117,117,0.441,0.336,1.383521,3.862665
118,0.220,118,118,0.401,0.195,1.291430,3.782758


In [1]:

if not subdirs:
    print("Empty output. Stopping processing")
    
rmses_year_day = np.zeros(len(subdirs))
rmses_day = np.zeros(len(subdirs))

for out in subdirs:
    ds_mod_ass = xr.open_dataset(os.path.join(base_path_output, str(out),"Q_ASSIMI_fluxnetdata_timestep.nc"))
    ds_mod_veg = xr.open_dataset(os.path.join(base_path_output,str(out), "VEG_fluxnetdata_timestep.nc"))
    ds_mod_sb  = xr.open_dataset(os.path.join(base_path_output,str(out), "SB_fluxnetdata_timestep.nc"))
    
    ...
    rmses_year_day[out] = vrmse_year_doy
    rmses_day[out] = vrmse_doy

NameError: name 'subdirs' is not defined